# Imports (compatible for Python 2 and Python 3)

In [1]:
# try:
#     import urllib2 as ulib
# except ImportError:
#     import urllib3 as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

In [2]:
API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

In [3]:
query = '''SELECT * FROM fb55.citibike;'''
table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0


In [7]:
query = '''SELECT * FROM fb55.citibike
WHERE tripduration > 1900
LIMIT 5;'''
table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,W 4 St & 7 Ave S,NaN,18,-74.002939,3589,2015-02-03 19:55:00+00,2015-02-03 20:55:00+00,380,11966,40.734011,441,E 52 St & 2 Ave,40.756014,-73.967416,18079,Customer,NaN,0
1,NaN,6 Ave & W 33 St,NaN,43,-73.988484,1906,2015-02-06 14:12:00+00,2015-02-06 14:43:00+00,505,34660,40.749013,173,Broadway & W 49 St,40.760647,-73.984427,17348,Customer,NaN,0
2,NaN,E 47 St & 1 Ave,NaN,53,-73.967844,2049,2015-02-07 10:35:00+00,2015-02-07 11:09:00+00,516,40705,40.752069,484,W 44 St & 5 Ave,40.755003,-73.980144,21066,Customer,NaN,0
3,NaN,Broadway & W 51 St,NaN,59,-73.983362,2092,2015-02-07 16:50:00+00,2015-02-07 17:25:00+00,500,43762,40.762288,447,8 Ave & W 52 St,40.763707,-73.985162,18605,Subscriber,1975.0,1
4,NaN,Grand Army Plaza & Central Park S,NaN,72,-73.973715,2474,2015-02-01 00:01:00+00,2015-02-01 00:42:00+00,281,3,40.764397,127,Barrow St & Hudson St,40.731724,-74.006744,18903,Subscriber,1969.0,2


In [8]:
query = '''SELECT MIN(tripduration) FROM fb55.citibike
LIMIT 5;'''
table = get_data(query)
table.head(5)

,min,Unnamed: 1
0,60,NaN


In [9]:
query = '''SELECT MAX(tripduration) - MIN(tripduration) as range FROM fb55.citibike
LIMIT 5;'''
table = get_data(query)
table.head(5)

,range,Unnamed: 1
0,42956,NaN


In [14]:
# for every bikeid, calculate the average tripduration
query = '''SELECT bikeid, AVG(tripduration) FROM fb55.citibike
GROUP BY bikeid
LIMIT 5;'''
table = get_data(query)
table.head(5)

,bikeid,avg
0,14923,902.000000
1,19505,562.000000
2,18996,606.571429
3,21027,709.631579
4,14963,716.916667


In [15]:
# select all avg tripdurion of all bikeid start from station 503
query = '''SELECT bikeid, AVG(tripduration) FROM fb55.citibike
WHERE start_station_id = 503
GROUP BY bikeid
LIMIT 5;'''
table = get_data(query)
table.head(5)

,bikeid,avg
0,14559,356.0
1,14592,985.0
2,14610,336.0
3,14692,481.0
4,14700,383.5


In [16]:
# query subquery
query = '''
SELECT * FROM

(SELECT bikeid, AVG(tripduration) FROM fb55.citibike
WHERE start_station_id = 503
GROUP BY bikeid
LIMIT 5) as subquery

WHERE bikeid = 14559
;'''
table = get_data(query)
table.head(5)

,bikeid,avg
0,14559,356.0


## Task 1.1

In [6]:
query = '''SELECT * FROM fb55.citibike ORDER BY start_station_id ASC, tripduration DESC;'''
table = get_data(query)
table.head(10)

,the_geom,start_station_longitude,the_geom_webmercator,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,cartodb_id,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,-73.978034,NaN,511,2/1/2015 0:50,2/1/2015 0:59,119,Park Ave & St Edwards St,40.696089,75,420,Clermont Ave & Lafayette Ave,40.687645,-73.969689,20792,Subscriber,1982.0,1
1,NaN,-74.002971,NaN,907,2/1/2015 0:17,2/1/2015 0:32,128,MacDougal St & Prince St,40.727103,26,531,Forsyth St & Broome St,40.718939,-73.992663,16917,Subscriber,1993.0,1
2,NaN,-74.002971,NaN,731,2/1/2015 1:11,2/1/2015 1:23,128,MacDougal St & Prince St,40.727103,92,502,Henry St & Grand St,40.714215,-73.981346,19506,Subscriber,1970.0,1
3,NaN,-74.002971,NaN,521,2/1/2015 2:29,2/1/2015 2:38,128,MacDougal St & Prince St,40.727103,166,276,Duane St & Greenwich St,40.717488,-74.010455,15023,Subscriber,1965.0,1
4,NaN,-74.002971,NaN,428,2/1/2015 1:54,2/1/2015 2:02,128,MacDougal St & Prince St,40.727103,142,312,Allen St & E Houston St,40.722055,-73.989111,21462,Subscriber,1983.0,1
5,NaN,-74.002971,NaN,224,2/1/2015 2:28,2/1/2015 2:32,128,MacDougal St & Prince St,40.727103,165,250,Lafayette St & Jersey St,40.724561,-73.995653,15715,Subscriber,1985.0,1
6,NaN,-73.980858,NaN,665,2/1/2015 0:53,2/1/2015 1:04,150,E 2 St & Avenue C,40.720874,81,497,E 17 St & Broadway,40.737050,-73.990093,18344,Subscriber,1984.0,1
7,NaN,-73.997203,NaN,763,2/1/2015 2:25,2/1/2015 2:38,151,Cleveland Pl & Spring St,40.721816,163,476,E 31 St & 3 Ave,40.743943,-73.979661,21554,Subscriber,1980.0,1
8,NaN,-73.997203,NaN,671,2/1/2015 0:58,2/1/2015 1:09,151,Cleveland Pl & Spring St,40.721816,87,433,E 13 St & Avenue A,40.729554,-73.980572,14706,Subscriber,1989.0,1
9,NaN,-73.997203,NaN,270,2/1/2015 1:22,2/1/2015 1:27,151,Cleveland Pl & Spring St,40.721816,106,2010,Grand St & Greene St,40.721655,-74.002347,19269,Subscriber,1980.0,1


In [10]:
query = '''SELECT * FROM fb55.citibike ORDER BY end_station_id DESC LIMIT 6;'''
table = get_data(query)
table.head(10)

,the_geom,start_station_longitude,the_geom_webmercator,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,cartodb_id,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,-73.991255,NaN,870,2/1/2015 1:34,2/1/2015 1:48,479,9 Ave & W 45 St,40.760193,120,2023,E 55 St & Lexington Ave,40.759681,-73.970314,18881,Subscriber,1985,2
1,NaN,-73.988484,NaN,417,2/1/2015 2:39,2/1/2015 2:46,505,6 Ave & W 33 St,40.749013,168,2021,W 45 St & 8 Ave,40.759291,-73.988597,18130,Subscriber,1967,1
2,NaN,-73.981656,NaN,234,2/1/2015 1:13,2/1/2015 1:17,504,1 Ave & E 15 St,40.732219,99,2012,E 27 St & 1 Ave,40.739445,-73.976806,15173,Subscriber,1984,1
3,NaN,-73.984267,NaN,419,2/1/2015 0:07,2/1/2015 0:14,326,E 11 St & 1 Ave,40.729538,14,2012,E 27 St & 1 Ave,40.739445,-73.976806,14679,Subscriber,1990,2
4,NaN,-73.994046,NaN,759,2/1/2015 0:04,2/1/2015 0:17,335,Washington Pl & Broadway,40.729039,9,2012,E 27 St & 1 Ave,40.739445,-73.976806,21183,Subscriber,1985,2
5,NaN,-73.975739,NaN,259,2/1/2015 0:24,2/1/2015 0:28,487,E 20 St & FDR Drive,40.733143,43,2012,E 27 St & 1 Ave,40.739445,-73.976806,16196,Subscriber,1990,1


## Task 1.2

In [53]:
# to select last 10 within SQL, order desc to sort it first
query = '''SELECT * FROM fb55.citibike
ORDER BY cartodb_id DESC
LIMIT 10;'''
table = get_data(query)
table.head(10)

,the_geom,start_station_longitude,the_geom_webmercator,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,cartodb_id,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,-73.976485,NaN,339,2/1/2015 3:47,2/1/2015 3:53,520,W 52 St & 5 Ave,40.759923,199,153,E 40 St & 5 Ave,40.752062,-73.981632,17389,Subscriber,1963,1
1,NaN,-73.977724,NaN,861,2/1/2015 3:45,2/1/2015 4:00,511,E 14 St & Avenue B,40.729387,198,335,Washington Pl & Broadway,40.729039,-73.994046,17927,Subscriber,1972,1
2,NaN,-73.999318,NaN,445,2/1/2015 3:44,2/1/2015 3:52,482,W 15 St & 7 Ave,40.739355,197,504,1 Ave & E 15 St,40.732219,-73.981656,18968,Subscriber,1990,1
3,NaN,-73.991255,NaN,412,2/1/2015 3:42,2/1/2015 3:49,479,9 Ave & W 45 St,40.760193,196,267,Broadway & W 36 St,40.750977,-73.987654,15185,Subscriber,1988,1
4,NaN,-74.003664,NaN,232,2/1/2015 3:41,2/1/2015 3:44,434,9 Ave & W 18 St,40.743174,195,494,W 26 St & 8 Ave,40.747348,-73.997236,17751,Subscriber,1975,1
5,NaN,-73.979955,NaN,401,2/1/2015 3:39,2/1/2015 3:46,393,E 5 St & Avenue C,40.722992,194,403,E 2 St & 2 Ave,40.725029,-73.990697,20358,Subscriber,1983,1
6,NaN,-73.950048,NaN,366,2/1/2015 3:33,2/1/2015 3:39,437,Macon St & Nostrand Ave,40.680983,193,344,Monroe St & Bedford Ave,40.685144,-73.953809,17284,Subscriber,1984,1
7,NaN,-73.988900,NaN,585,2/1/2015 3:32,2/1/2015 3:41,483,E 12 St & 3 Ave,40.732233,192,268,Howard St & Centre St,40.719105,-73.999733,21046,Subscriber,1994,1
8,NaN,-73.981948,NaN,148,2/1/2015 3:29,2/1/2015 3:32,472,E 32 St & Park Ave,40.745712,191,472,E 32 St & Park Ave,40.745712,-73.981948,20727,Subscriber,1982,1
9,NaN,-74.002638,NaN,398,2/1/2015 3:28,2/1/2015 3:35,284,Greenwich Ave & 8 Ave,40.739017,190,285,Broadway & E 14 St,40.734546,-73.990741,19040,Subscriber,1977,1


## Task 1.3

In [11]:
query = '''SELECT DISTINCT birth_year FROM fb55.citibike 
WHERE birth_year IS NOT NULL;'''
table = get_data(query)
table.head(10)

,birth_year,Unnamed: 1
0,1970,NaN
1,1978,NaN
2,1960,NaN
3,1987,NaN
4,1994,NaN
5,1965,NaN
6,1977,NaN
7,1973,NaN
8,1995,NaN
9,1990,NaN


In [15]:
# find the oldest person on the go
query = '''SELECT 2018-MIN(birth_year) FROM fb55.citibike;'''
table = get_data(query)
table.head(5)

,?column?,Unnamed: 1
0,64,NaN


## Task 1.4

In [38]:
query = '''SELECT MIN(tripduration) as TripDdduration FROM fb55.citibike;'''
table = get_data(query)
table.head(5)

,tripddduration,Unnamed: 1
0,89,NaN


In [21]:
query = '''SELECT MIN(tripduration), MAX(tripduration) FROM fb55.citibike;'''
table = get_data(query)
table.head(5)

,min,max
0,89,35251


In [25]:
query = '''SELECT MIN(tripduration) FROM fb55.citibike;'''
table = get_data(query)
print(table)

query = '''SELECT MAX(tripduration) FROM fb55.citibike;'''
table = get_data(query)
print(table)

query = '''SELECT AVG(tripduration) FROM fb55.citibike;'''
table = get_data(query)
print(table)

   min  Unnamed: 1
0   89         NaN
     max  Unnamed: 1
0  35251         NaN
          avg  Unnamed: 1
0  775.527638         NaN


## Task 2.1

In [27]:
query = '''SELECT bikeid FROM fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time)=1;'''
table = get_data(query)
table.head(5)

,bikeid,Unnamed: 1
0,20142,NaN
1,20684,NaN
2,15792,NaN
3,19506,NaN
4,15185,NaN


In [29]:
# midnight 1am and 2am
query = '''SELECT bikeid FROM fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time) IN (0, 1, 2);'''
table = get_data(query)
table.head(5)

,bikeid,Unnamed: 1
0,17131,NaN
1,20142,NaN
2,21289,NaN
3,18903,NaN
4,21044,NaN


## Task 2.2

In [32]:
query = '''SELECT MAX(2018-birth_year) FROM fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time)=3;'''
table = get_data(query)
table.head(5)

,max,Unnamed: 1
0,64,NaN


In [34]:
query = '''SELECT AVG(birth_year) FROM fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time)=2;'''
table = get_data(query)
table.head(5)

,avg,Unnamed: 1
0,1981.129032,NaN


## Task 3.1

In [52]:
# COUNT(bikeid) gives how many bikes were taken, unique bikes is not counted twice, COUNT(start_station_id) gives how many total takes, bikes might be taken multiple times
query = '''SELECT start_station_id, COUNT(bikeid) as count FROM fb55.citibike
GROUP BY start_station_id
ORDER BY count DESC
LIMIT 3;'''
table = get_data(query)
table.head(5)

,start_station_id,count
0,285,6
1,250,6
2,401,5


## Task 3.2

In [47]:
query = '''SELECT end_station_id, SUM(tripduration) as duration FROM fb55.citibike
GROUP BY end_station_id
ORDER BY duration DESC
LIMIT 3;'''
table = get_data(query)
table.head(5)

,end_station_id,duration
0,525,35251
1,237,4297
2,127,3806


## Task 3.3

In [48]:
query = '''SELECT start_station_id, AVG(tripduration) as avg FROM fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time)=1
GROUP BY start_station_id
ORDER BY avg ASC;'''
table = get_data(query)
table.head(5)

,start_station_id,avg
0,486,114.0
1,325,155.0
2,345,164.0
3,449,181.0
4,237,250.0
